In [1]:
import pandas as pd
import numpy as np
import json, re

# Torch, Sklearn imports
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
print(torch.__version__)

## Embeddings
import allennlp
from allennlp.modules.elmo import Elmo, batch_to_ids
from gensim.models import KeyedVectors

## NLP libs
from nltk import download
import gensim
from nltk.corpus import stopwords
download('stopwords')

0.4.1
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rsilvei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Importing SNIPS intent dataset

In [2]:
dataset = pd.DataFrame(columns = ['phrase', 'intent'])

In [3]:
for intent in ['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork',
               'SearchScreeningEvent']:
    with open("./2017-06-custom-intent-engines/" + intent + "/train_" + intent + ".json",
              encoding='cp1251') as data_file:
        data = json.load(data_file)
    print("Intent: {}, Length: {}".format(intent,len(data[intent])))
    texts = []
    for i in range(len(data[intent])):
        text = ''
        for j in range(len(data[intent][i]['data'])):
            text += data[intent][i]['data'][j]['text']
        dataset = dataset.append({'phrase': text, 'intent': intent}, ignore_index=True)

Intent: AddToPlaylist, Length: 300
Intent: BookRestaurant, Length: 300
Intent: GetWeather, Length: 300
Intent: PlayMusic, Length: 300
Intent: RateBook, Length: 300
Intent: SearchCreativeWork, Length: 300
Intent: SearchScreeningEvent, Length: 300


In [4]:
dataset.intent.unique()

array(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic',
       'RateBook', 'SearchCreativeWork', 'SearchScreeningEvent'],
      dtype=object)

In [5]:
def transformText(text, do_stop=False, do_stem=False):
    stops = set(stopwords.words("english"))
    # Convert text to lower
    text = text.lower()
    
    # Cleaning input
    text = text.replace("'s","")
    text = text.replace("’s","")
    text = text.replace("?","")
    text = text.replace("-","")
    
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    # Removing all the stopwords
    if (do_stop==True):
        filtered_words = [word for word in text.split() if word not in stops]
    else:
        filtered_words = [word for word in text.split()]
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    # Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    if (do_stem==True):
        # Stemming
        text = gensim.parsing.preprocessing.stem_text(text)
    return text

def strip_punctuation(s):
    return ''.join(c for c in s if c not in PUNCT)

## Lemmatization function based on Spacy Library
def lemmatizer_spacy(text):        
    sent = []
    doc = spacy_en(text)
    for word in doc:
        if word.lemma_ == "-PRON-":
            sent.append(word.text)
        else:
            sent.append(word.lemma_)
    return " ".join(sent)

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [6]:
dataset['preproc_text'] = dataset['phrase'].apply(lambda x: transformText(x))

In [7]:
dataset.tail(10)

,phrase,intent,preproc_text
2090,I want to see a list of the closest cinema's m...,SearchScreeningEvent,i want to see a list of the closest cinema movies
2091,What Are the showings for The Natural History ...,SearchScreeningEvent,what are the showings for the natural history ...
2092,Give me the schedule for Public Stenographer a...,SearchScreeningEvent,give me the schedule for public stenographer a...
2093,Is it possible to see Tube at the closest movi...,SearchScreeningEvent,is it possible to see tube at the closest movi...
2094,I want to see Wenn Lucy springt now at a movie...,SearchScreeningEvent,i want to see wenn lucy springt now at a movie...
2095,Is Across the Line playing at the closest movi...,SearchScreeningEvent,is across the line playing at the closest movi...
2096,Which animated movies are playing in the neigh...,SearchScreeningEvent,which animated movies are playing in the neigh...
2097,Where is They Always Return at Dawn playing,SearchScreeningEvent,where is they always return at dawn playing
2098,What is the movie schedule in the neighborhood,SearchScreeningEvent,what is the movie schedule in the neighborhood
2099,Tell me when Howling II: Your Sister Is a Were...,SearchScreeningEvent,tell me when howling ii your sister is a werew...


In [8]:
!ls ../../../vectors/

GoogleNews-vectors-negative300.bin
crawl-300d-2M.vec
elmo_2x1024_128_2048cnn_1xhighway_options.json
elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5
elmo_2x2048_256_2048cnn_1xhighway_options.json
elmo_2x2048_256_2048cnn_1xhighway_weights.hdf5
elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json
elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5
elmo_2x4096_512_2048cnn_2xhighway_options.json
elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5
glove.42B.300d.txt
glove.840B.300d.txt
lid.176.ftz
wiki-news-300d-1M-subword.txt
wiki-news-300d-1M.txt


## Build Vocabulary

In [9]:
## Build word vocabulary
word_to_ix = {}
for sent in dataset.preproc_text:
    for word in sent.split():
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print("Tamanho do dicionario: {}".format(len(word_to_ix)))

Tamanho do dicionario: 3440


In [10]:
## Build label vocabulary
label_to_ix = {}
for label in dataset.intent:
    for word in label.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)

In [11]:
label_to_ix

{'AddToPlaylist': 0,
 'BookRestaurant': 1,
 'GetWeather': 2,
 'PlayMusic': 3,
 'RateBook': 4,
 'SearchCreativeWork': 5,
 'SearchScreeningEvent': 6}

## Preparing PyTorch Dataset Loader

In [12]:
class Intents(Dataset):
    def __init__(self, dataframe, w2v_weights_path):
        self.len = len(dataframe)
        self.label_to_ix = {}
        self.data = dataframe
        self.w2v = w2v = KeyedVectors.load_word2vec_format(w2v_weights_path, binary = True)
        
    def __getitem__(self, index):
        phrase = self.data.preproc_text[index]
        X, _  = self.get_avg_sentence_vector(phrase)
        y = label_to_ix[self.data.intent[index]]
        #X.requires_grad = False
        #y.requires_grad = False
        #blin = X.detach()
        return X, y
    
    def __len__(self):
        return self.len

    def get_avg_sentence_vector(self, sentence):
        featureVec = np.zeros((self.w2v.vector_size), dtype="float32")
        nwords = 0
        not_found_words = []
        for word in sentence.split():
            if word in self.w2v.index2word:
                nwords = nwords+1
                featureVec = np.add(featureVec, self.w2v.get_vector(word))
            else:
                not_found_words.append(word)
        if nwords>0:
            featureVec = np.divide(featureVec, nwords)
        return featureVec, not_found_words

In [13]:
# Set data locations for embeddings
elmo_config_key_path = '../../../vectors/elmo_2x4096_512_2048cnn_2xhighway_options.json'
elmo_weights_key_path = '../../../vectors/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5'
w2v_weights_path = '../../../vectors/GoogleNews-vectors-negative300.bin'

In [14]:
train_size = 0.8
train_dataset=dataset.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_dataset=dataset.drop(train_dataset.index).reset_index(drop=True)

In [15]:
print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (2100, 3)
TRAIN Dataset: (1680, 3)
TEST Dataset: (420, 3)


In [16]:
training_set = Intents(train_dataset,  w2v_weights_path)
testing_set = Intents(test_dataset, w2v_weights_path)

In [17]:
testing_set.__getitem__(3)

(array([ 1.66388620e-02,  5.91905378e-02,  2.37630215e-02,  6.19167760e-02,
         2.11588549e-03,  2.12198887e-02,  1.03074819e-01, -7.42187500e-02,
         4.19116542e-02,  4.27110456e-02, -6.72844797e-02, -8.44997838e-02,
        -3.00021693e-02,  1.39024518e-02, -1.12277560e-01,  1.00914851e-01,
         9.73205566e-02,  9.18782577e-02,  4.77447510e-02, -1.48383249e-02,
        -2.03762054e-02,  2.84016933e-02, -6.75591379e-02, -8.17362498e-03,
         5.12152761e-02, -3.71636301e-02, -5.46875000e-02,  5.25037982e-02,
        -1.78019202e-03, -1.38888890e-02, -4.50303815e-02,  2.45700404e-02,
        -1.99924037e-02,  7.05634207e-02,  7.95084611e-02,  2.36100089e-02,
         7.47477189e-02, -3.26605886e-02,  7.50630721e-02,  3.38812917e-02,
         9.62320939e-02,  2.27050781e-02,  5.80105260e-02, -4.38944511e-02,
        -1.66558158e-02,  4.10291890e-04, -4.19040248e-02,  2.22066250e-02,
         7.48697901e-03,  7.97526073e-03, -5.54334857e-02,  3.22672538e-02,
         2.3

## Simple MLP Classifier

In [18]:
class SimpleMLP(nn.Module):
    def __init__(self, inputdim, 
                        nclasses, 
                        nhidden, 
                        dropout = 0,
                        cudaEfficient=True):
        super(SimpleMLP, self).__init__()
        """
        PARAMETERS:
        -dropout:    dropout for MLP
        """
        
        self.inputdim = inputdim
        self.hidden_dim = nhidden
        self.dropout = dropout
        self.nclasses = nclasses
        
        if cudaEfficient:
            self.model = nn.Sequential(
                nn.Linear(self.inputdim, nhidden),
                nn.Dropout(p=self.dropout),
                nn.ReLU(),
                nn.Linear(nhidden, self.nclasses),
                ).cuda()
        else:
            self.model = nn.Sequential(
                nn.Linear(self.inputdim, nhidden),
                nn.Dropout(p=self.dropout),
                nn.ReLU(),
                nn.Linear(nhidden, self.nclasses),
                )
    def forward(self, x):
        log_probs = self.model(x)
        return log_probs

In [19]:
INP_DIM = training_set.w2v.vector_size
NUM_LABELS = len(label_to_ix)
NHIDDEN = 512
DROPOUT = 0.3
model = SimpleMLP(inputdim = INP_DIM ,
          nhidden = NHIDDEN,
          nclasses = NUM_LABELS,
          dropout = DROPOUT, 
          cudaEfficient = False)

In [20]:
NUM_LABELS

7

## Training

In [21]:
# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 1}

In [22]:
training_loader = DataLoader(training_set, **params)
testing_loader = DataLoader(testing_set, **params)

In [23]:
training_loader.batch_size

64

In [24]:
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)

In [25]:
max_epochs = 10
for epoch in range(max_epochs):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        ## Step 1 - Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        sent = Variable(sent)
        label = Variable(label)
        
        ## Step 2 - Run forward pass
        output = model.forward(sent)
        
        # Get predictions from the maximum value
        _, predicted = torch.max(output.data, 1)
        
        ## Step 3 - Compute loss
        loss = loss_function(output, label)
        loss.backward()
        
        ## Step 4 = Update parameters
        optimizer.step()
        
        if i%100 == 0:
            
            # Calculate Accuracy         
            correct = 0
            total = 0
            
            for sent, label in testing_loader:
                sent = Variable(sent)
                label = Variable(label)
                
                # Forward pass only to get logits/output
                output = model.forward(sent)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(output.data, 1)
                
                # Total number of labels
                total += label.size(0)

                # Total correct predictions
                correct += (predicted.cpu() == label.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.data[0], accuracy))

EPOCH -- 0


/Users/rsilvei/Envs/nlp_new/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iteration: 0. Loss: 1.942359447479248. Accuracy: 0.47619047619047616%
EPOCH -- 1
Iteration: 0. Loss: 1.3663665056228638. Accuracy: 80.47619047619048%
EPOCH -- 2
Iteration: 0. Loss: 0.6236667633056641. Accuracy: 85.71428571428571%
EPOCH -- 3
Iteration: 0. Loss: 0.2919796407222748. Accuracy: 91.66666666666667%
EPOCH -- 4
Iteration: 0. Loss: 0.2598581314086914. Accuracy: 93.33333333333333%
EPOCH -- 5
Iteration: 0. Loss: 0.28362828493118286. Accuracy: 94.76190476190476%
EPOCH -- 6
Iteration: 0. Loss: 0.08243129402399063. Accuracy: 96.66666666666667%
EPOCH -- 7
Iteration: 0. Loss: 0.08112271130084991. Accuracy: 97.61904761904762%
EPOCH -- 8
Iteration: 0. Loss: 0.06717398762702942. Accuracy: 97.38095238095238%
EPOCH -- 9
Iteration: 0. Loss: 0.06588076800107956. Accuracy: 98.57142857142857%


In [26]:
input_phrase = "i need to book a restaurant today"

In [27]:
def get_reply(phrase):
    inp, _ = training_set.get_avg_sentence_vector(phrase)
    inp = Variable(torch.Tensor(inp))
    output = model.forward(inp)

    # Get predictions from the maximum value
    _, predicted = torch.max(output.data, 0)
    pred_label=list(label_to_ix.keys())[list(label_to_ix.values()).index(predicted.item())]
    return pred_label

In [28]:
get_reply(input_phrase)

'BookRestaurant'

In [29]:
label_to_ix

{'AddToPlaylist': 0,
 'BookRestaurant': 1,
 'GetWeather': 2,
 'PlayMusic': 3,
 'RateBook': 4,
 'SearchCreativeWork': 5,
 'SearchScreeningEvent': 6}

## Intent Classifier w/ Uncertainty - MC Dropout

In [94]:
class MLPUncertainty(nn.Module):
    def __init__(self, inputdim, 
                        nclasses, 
                        nhidden, 
                        dropout = 0,
                        cudaEfficient=True,
                        decay = 1e-6):
        super(MLPUncertainty, self).__init__()
        """
        PARAMETERS:
        -dropout:    dropout for MLP
        """
        
        self.inputdim = inputdim
        self.hidden_dim = nhidden
        self.dropout = dropout
        self.decay = decay
        self.nclasses = nclasses
        
        if cudaEfficient:
            self.model = nn.Sequential(
                nn.Linear(self.inputdim, nhidden),
                nn.Dropout(p=self.dropout),
                nn.ReLU(),
                nn.Linear(nhidden, self.nclasses),
                ).cuda()
        else:
            self.model = nn.Sequential(
                nn.Linear(self.inputdim, nhidden),
                nn.Dropout(p=self.dropout),
                nn.ReLU(),
                nn.Linear(nhidden, self.nclasses),
                )
    def forward(self, x):
        log_probs = self.model(x)
        return log_probs

In [95]:
def uncertainity_estimate(x, model, iters, l2):
    outputs = np.hstack([model(x).data.numpy() for i in trange(iters)])
    y_mean = outputs.mean(axis=1)
    y_variance = outputs.var(axis=1)
    tau = l2 * (1. - model.dropout_p) / (2. * N * model.decay)
    y_variance += (1. / tau)
    y_std = np.sqrt(y_variance)
    return y_mean, y_std

In [97]:
x_test = 'play this music please'

In [98]:
from tqdm import trange

In [100]:
get_reply(x_test)

'PlayMusic'

In [101]:
inp, _ = training_set.get_avg_sentence_vector(phrase)
inp = Variable(torch.Tensor(inp))
output = model_uncertainty.forward(inp).data.numpy()
output

array([-0.0557947 , -0.01062911, -0.0246579 , -0.00250984,  0.00657175,
        0.04260978,  0.02417766], dtype=float32)

In [485]:
iters_uncertainty = 20
lengthscale = 0.01
N = 1 

In [478]:
def get_reply_test(phrase):
    inp, _ = training_set.get_avg_sentence_vector(phrase)
    inp = Variable(torch.Tensor(inp))
    output = model.forward(inp)
    #_, predicted = torch.max(output.data, 0)
    output = F.softmax(output, dim=0)
    return output

In [492]:
y_mean.data

In [497]:
np.argmax(y_mean)

6

In [493]:
torch.max(y_mean, 0)

TypeError: max() received an invalid combination of arguments - got (numpy.ndarray, int), but expected one of:
 * (Tensor input)
 * (Tensor input, Tensor other, Tensor out)
 * (Tensor input, int dim, bool keepdim, tuple of Tensors out)


In [500]:
def get_reply_uncertainty(phrase, iters_uncertainty, lengthscale, N):
    outputs = [get_reply_test(phrase).data.numpy() for i in range(iters_uncertainty)]
    outs = np.vstack(outputs)
    y_mean = outs.mean(axis=0)
    y_variance = outs.var(axis=0).sum()
    tau = lengthscale * (1. - model_uncertainty.dropout) / (2. * N * model_uncertainty.decay)
    y_variance += (1. / tau)
    y_std = np.sqrt(y_variance)
    predicted = np.argmax(y_mean)
    pred_label=list(label_to_ix.keys())[list(label_to_ix.values()).index(predicted)]
    return pred_label, y_std

In [507]:
#phrase = 'trump is idiot'
x_test = ['trump is idiot',
          'when is warm',
          'play lady gaga music',
          'give 5 stars to machine learning book',
          'play reaggae', 
          'add songs tune in reggae infusions',
          'give 5 start to this novel',
          'barack obama is cool',
          'christian perone is a great software engineer',
          'thomas paula is giving unsupervised learning classes',
          'paçoca likes candy he is our IT guy'
         ]
x_test_2 = [
    'give 5 stars to machine learning book',
    'christian is going to NIPS',
    'play blues songs',
    'today is friday',
    'mexicans eat nachos'
]
for utt in x_test_2:
    predicted, y_std = get_reply_uncertainty(utt, iters_uncertainty, lengthscale, N)
    print("[INPUT]: {}".format(utt))
    if y_std > 0.05:
        print("[UNCERTAIN] I'm not sure, lot's of variance: Std {}".format(y_std))
    else:
        print("[PREDICTED LABEL]: {}".format(predicted))
        print("[STD DEV]: {}".format(y_std))
    print("-"*50)

[INPUT]: give 5 stars to machine learning book
[PREDICTED LABEL]: RateBook
[STD DEV]: 0.017623399787340167
--------------------------------------------------
[INPUT]: christian is going to NIPS
[UNCERTAIN] I'm not sure, lot's of variance: Std 0.06501580313801136
--------------------------------------------------
[INPUT]: play blues songs
[PREDICTED LABEL]: PlayMusic
[STD DEV]: 0.017245198745267473
--------------------------------------------------
[INPUT]: today is friday
[UNCERTAIN] I'm not sure, lot's of variance: Std 0.13220545720844648
--------------------------------------------------
[INPUT]: mexicans eat nachos
[PREDICTED LABEL]: BookRestaurant
[STD DEV]: 0.021249837530390555
--------------------------------------------------


In [337]:
label_to_ix

{'AddToPlaylist': 0,
 'BookRestaurant': 1,
 'GetWeather': 2,
 'PlayMusic': 3,
 'RateBook': 4,
 'SearchCreativeWork': 5,
 'SearchScreeningEvent': 6}

In [505]:
train_dataset.tail(20)

,phrase,intent,preproc_text
1660,What times will Escape from Hong Kong be showing?,SearchScreeningEvent,what times will escape from hong kong be showing
1661,most popular song of taco ockerse,PlayMusic,most popular song of taco ockerse
1662,rate this current series zero out of 6,RateBook,rate this current series zero out of 6
1663,Put some fifties music on Netflix,PlayMusic,put some fifties music on netflix
1664,What will the weather be here at twelve P.m.?,GetWeather,what will the weather be here at twelve p m
1665,Can you play some music by Abatte Barihun?,PlayMusic,can you play some music by abatte barihun
1666,rate the current essay 1 out of 6 stars,RateBook,rate the current essay 1 out of 6 stars
1667,give the current textbook 1 out of 6 stars,RateBook,give the current textbook 1 out of 6 stars
1668,Play chant music from 2016 off Pandora.,PlayMusic,play chant music from 2016 off pandora
1669,Play a sound track from 1952.,PlayMusic,play a sound track from 1952


## Intent Classifier w/ Uncertainty - Deep Ensembles